# Correlation of COVID-19 Infection Rates with Happiness of a Country

In this project, we will be looking for the corrleation between the infection rate of COVID-19 from around the countries around the world, with the happiness of the country's citizen. Correlation is measure of the strength of a linear relationship between two quantitative variables. For the infection rate, we will be using data of COVID infecton from the Global Convirmed Case by the John Hopkins University, and for the happiness of the countries, we will be using The World Happiness Report from the Gallup World Poll

Now since this is a broad subject, we will need to create a limitation for our analysis. For the Infection Rate of COVID-19, we will be using metrics from the Average of Infection per Day and Highest Infection Per Day, from the start of the pandemic untill present days. And for the Happiness Metrics, we will be using Ladder Score, which is the happiness score for a country. We would also create comparison from other metrics in the World Happiness Report with infection rate of COVID. 

First, we need to input the libraries for our analysis. 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

We import 4 libraries for our analysis, pandas, numpy, matplotlib, and seaborn. 

pandas is a software library written for the Python programming language for data manipulation and analysis.

NumPy is a library for the Python programming language, adding support for large, multi-dimensional arrays and matrices.

matplotlib and seaborn, is a visualization library to help us visualize the result of our analysis. 

In [ ]:
path_corona = '../input/covid19-data-from-john-hopkins-university/RAW_global_confirmed_cases.csv'
corona_dataset_csv = pd.read_csv(path_corona, delimiter=',') 
corona_dataset_csv.head()

In [ ]:
corona_dataset_csv.shape

After importing our libraries, now we will be importing our dataset. First, we will import the dataset for COVID-19 Data from John Hopkins University. The dataset consist of Country/Region, Priovince/State, Latitude & Longitude of the countries, and the number of cases of COVID-19 from January 22th of 2020 untill September 8th of 2021 from each countries.

From the shape of our dataset, we can see that we have 279 row, but since the some countries are split between their provinces, we will need to merge their data to get the total infection of each countries per day. 

Also, the latitude and longitude does not contribute to our analysis, which is why we will be dropping the column.

In [ ]:
corona_dataset_csv.drop(['Lat', 'Long'], axis=1, inplace=True)
corona_dataset_csv.head()

In [ ]:
corona_dataset_aggregated = corona_dataset_csv.groupby('Country/Region').sum()
corona_dataset_aggregated.head()

In [ ]:
corona_dataset_aggregated.shape

After manipulating our data, we now have dropped the column Lat and Long, and also we have group our data so that the infection of each countries are totaled.

After aggregating the infection rates from each country, we now have a data from 195 countries, and their dialy infection rate. To better understand about the infection rate, we will be visualizing infection rate from 5 sample countries, Indonesia, Italy, United States, Australia, and South Africa. 

In [ ]:
corona_dataset_aggregated.loc['Indonesia'].plot()
corona_dataset_aggregated.loc['Italy'].plot()
corona_dataset_aggregated.loc['Canada'].plot()
corona_dataset_aggregated.loc['Australia'].plot()
corona_dataset_aggregated.loc['South Africa'].plot()
plt.legend()
plt.title('Infection Cases of COVID-19')

From the visualization above, we can see how the number of recorded COVID-19 cases climbed up. 

But for our analysis, we will need to process the data, since we want to know how much is the highest climb active cases dialy from each countries, and the average increase of active cases dialy. We will make two new calculations. 

In [ ]:
corona_dataset_aggregated.loc['US'].diff().max()

In [ ]:
corona_dataset_aggregated.loc['US'].diff().mean()

Above, we use US as a sample country. We diff() to find the changes of cases from each day, use max() to find the highest dialy change of infection, and mean() to find the average dialy changes of infection.

Now that we have the calculation for our analysis, we need to apply them for every countries in the list. 

In [ ]:
#Find maximum infection rate for all of the countries
countries = list(corona_dataset_aggregated.index)
max_infection_rates = []
for c in countries : 
  max_infection_rates.append(corona_dataset_aggregated.loc[c].diff().max())

avg_infection_rates = []
for d in countries : 
  avg_infection_rates.append(corona_dataset_aggregated.loc[d].diff().mean())

corona_dataset_aggregated['max_infection_rates'] = max_infection_rates
corona_dataset_aggregated['avg_infection_rates'] = avg_infection_rates
corona_dataset_aggregated.head() #Combine max infection rate to the dataset

Now, we have the max and average infection from each countries, therefore no longer needs the dialy cases data from our dataset. 

Therefore, we will be dropping the dialy cases from our dataset, leaving only country, max_infection_rates, and avg_infection_rates.

In [ ]:
corona_dataset_aggregated

In [ ]:
important_data = ['max_infection_rates', 'avg_infection_rates']

corona_data = pd.DataFrame(corona_dataset_aggregated[important_data])
corona_data.head()

Now that we have the data for our Infection Rate, we need the data for our countries Happiness. 

First, we load the dataset using pandas. 

In [ ]:
#Load happiness dataset
path = '../input/world-happiness-report-2021/world-happiness-report-2021.csv'
happiness_report_csv = pd.read_csv(path, delimiter=',') 
happiness_report_csv.head()

After seeing the dataset, we notice that there are several data that we do not need, such as regional indicator, standard error of ladder score, and etc. 

Therefore, our first step is to remove the unnecessary column. To do that, we first list the name of the unnecessary column for our analysis, and put them inside a variable. Then all we have to do is use the drop() function to remove all the column that we have listed before. 

In [ ]:
remove_cols = ['Regional indicator', 'Standard error of ladder score', 'upperwhisker', 'lowerwhisker', 'Generosity', 'Perceptions of corruption', 'Ladder score in Dystopia', 'Explained by: Log GDP per capita', 'Explained by: Social support', 'Explained by: Healthy life expectancy', 'Explained by: Freedom to make life choices', 'Explained by: Generosity', 'Explained by: Perceptions of corruption', 'Dystopia + residual']

In [ ]:
happiness_report_csv.drop(happiness_report_csv[remove_cols], axis=1, inplace=True)
happiness_report_csv.head()

Now that the dataset only consist of important data to us, we need to merge the happiness dataset, with the previous dataset that we made about infection rate. To do that, we need to change something first in our happiness dataset. 

In our happiness dataset, the index of the dataset is still ordinal numbers. To allow us to join the two dataset, we need to change the index into countries first. 

In [ ]:
happiness_report_csv.set_index('Country name', inplace=True)
happiness_report_csv.head()

Now that we have change the index, we can continue to join the two dataset. 

In [ ]:
corona_data.shape

In [ ]:
happiness_report_csv.shape

In [ ]:
data = corona_data.join(happiness_report_csv, how='inner')
data.head()

See that the happiness_report_csv, have less row than corona_data. This mean that not all of the countries in corona_data is listed in happiness_report_csv. 

Since we only need the list of countries that is in both of the dataset, we join them using inner join. Now that we have the data we need, we can start our analysis. 

In [ ]:
data.corr()

Since our analysis focused on the correlation between the infection rates and happiness score, we created a correlation matrix between each variable. closer to 1 mean that there is a positive correlation, closer to 0 mean there is no corrlelation, and closer to -1 mean that there is a negative correlation.

But we can't understand this data. We need to visualize it to be able to understand it better. We will start with simple visualization between infection rates and ladder score. 

In [ ]:
infection_max = data['max_infection_rates']
infection_avg = data['avg_infection_rates']
ladder = data['Ladder score']
gdp = data['Logged GDP per capita']
social = data['Social support']
health = data['Healthy life expectancy']
freedom = data['Freedom to make life choices']

fig, axs = plt.subplots(2)
sns.regplot(x=ladder, y=infection_max, ax=axs[0])
sns.regplot(x=ladder, y=infection_avg, ax=axs[1])
fig.suptitle('Correlation between Infection Rates and Ladder Score', fontsize = 30, y=1)  

We created our plot, but the data points is too close to each other to create any kind of conclusions. To better understand this data, we will use numpy np.log() function.

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))
sns.regplot(x=ladder, y=np.log(infection_max), ax=axs[0])
sns.regplot(x=ladder, y=np.log(infection_avg), ax=axs[1])
fig.suptitle('Correlation between Infection Rates and Ladder Score', fontsize = 15, y=1)  

Now our data is easier to see, and easier to read. Before we draw to any conclusions, we will also visualize the correlation between infection rates and the rest of the happiness report metrics. 

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))
sns.regplot(x=gdp, y=np.log(infection_max), ax=axs[0])
sns.regplot(x=gdp, y=np.log(infection_avg), ax=axs[1])
fig.suptitle('Correlation between Infection Rates and GDP Per Capita', fontsize = 15, y=1) 

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))
sns.regplot(x=social, y=np.log(infection_max), ax=axs[0])
sns.regplot(x=social, y=np.log(infection_avg), ax=axs[1])
fig.suptitle('Correlation between Infection Rates and Social Support', fontsize = 15, y=1) 

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))
sns.regplot(x=health, y=np.log(infection_max), ax=axs[0])
sns.regplot(x=health, y=np.log(infection_avg), ax=axs[1])
fig.suptitle('Correlation between Infection Rates and Health Life Expectancy', fontsize = 15, y=1) 

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))
sns.regplot(x=freedom, y=np.log(infection_max), ax=axs[0])
sns.regplot(x=freedom, y=np.log(infection_avg), ax=axs[1])
fig.suptitle('Correlation between Infection Rates and Freedom', fontsize = 15, y=1) 

In our analysis, we compare highest infection rate dialy, and average infection dialy, with the 5 metrics of happines from countries around the globe. From our visualization, we can see that there is a positive correlation between the infection rates, and the happines from each countries. 

To put it simply, a more happy and stable country, appear to have a higher infection rates than others. Of course there is many factors and variable that could effect this, and this is just a simple metrics that visualize the factors of infection rates from each countries. 

Therefore, based on the data that we have, a happier country have a higher infection rate than less happier countries. 